In [400]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor
pd.set_option("display.max_columns",1000000)
pd.set_option('display.max_rows', 1000000)

In [401]:
train_data = pd.read_csv('./new_train.csv')
remastered_train = pd.read_csv('./remastered.csv')


In [402]:
remastered_train.drop(['index','Id'],axis=1,inplace=True)

In [403]:
def transform(X):
    X["TotalHouse"] = X["TotalBsmtSF"] + X["1stFlrSF"] + X["2ndFlrSF"]   
    X["TotalArea"] = X["TotalBsmtSF"] + X["1stFlrSF"] + X["2ndFlrSF"] + X["GarageArea"]

    X["+_TotalHouse_OverallQual"] = X["TotalHouse"] * X["OverallQual"]
    X["+_GrLivArea_OverallQual"] = X["GrLivArea"] * X["OverallQual"]


    X["+_BsmtFinSF1_OverallQual"] = X["BsmtFinSF1"] * X["OverallQual"]

    X["-_Functional_TotalHouse"] = X["Functional"] * X["TotalHouse"]
    X["-_Functional_OverallQual"] = X["Functional"] + X["OverallQual"]
    X["-_LotArea_OverallQual"] = X["LotArea"] * X["OverallQual"]
    X["-_TotalHouse_LotArea"] = X["TotalHouse"] + X["LotArea"]



    X["Bsmt"] = X["BsmtFinSF1"] + X["BsmtFinSF2"] + X["BsmtUnfSF"]
    X["Rooms"] = X["FullBath"]+X["TotRmsAbvGrd"]
    X["PorchArea"] = X["OpenPorchSF"]+X["EnclosedPorch"]+X["3SsnPorch"]+X["ScreenPorch"]
    X["TotalPlace"] = X["TotalBsmtSF"] + X["1stFlrSF"] + X["2ndFlrSF"] + X["GarageArea"] + X["OpenPorchSF"]+X["EnclosedPorch"]+X["3SsnPorch"]+X["ScreenPorch"]


    return X

In [404]:
train_data = transform(train_data)

In [405]:
droplist = ['TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea','OverallQual','BsmtFinSF1','Functional','LotArea','BsmtFinSF2','BsmtUnfSF','BsmtFinSF1','FullBath','TotRmsAbvGrd','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','TotalBsmtSF']

In [406]:
# train_data.drop(droplist,axis = 1,inplace = True)

In [407]:
train_data.shape

(1460, 210)

In [408]:
scaler = RobustScaler()

In [409]:
train_data_X = train_data.drop('SalePrice',axis = 1)
train_data_y = train_data['SalePrice']
log_train_y = np.log(train_data_y)

In [410]:
for i in train_data_X.columns:
    if 'SaleCondition' in i:
        train_data_X.drop(i,axis = 1,inplace = True)

In [411]:
train_X_scaled = scaler.fit(train_data_X).transform(train_data_X)

In [412]:
pca_train = PCA(n_components=50,svd_solver='full')

In [413]:
train_X_after_PCA = pca_train.fit_transform(train_X_scaled)

In [414]:
normal_index_list = list(remastered_train[remastered_train['SaleCondition'] == 'Normal'].index)

In [415]:
train_X_after_PCA = pd.DataFrame(train_X_after_PCA)

In [416]:
train_data_for_XGboost = train_X_after_PCA.ix[normal_index_list]

/Users/hongzhengnan/anaconda3/envs/dme/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [417]:
type(train_X_after_PCA)

pandas.core.frame.DataFrame

In [418]:
xgboost = XGBRegressor()
xgboost.fit(np.array(train_X_after_PCA),log_train_y)

/Users/hongzhengnan/Desktop/dme/house-price/xgboost/python-package/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[23:24:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [419]:
#price_origin_train =pd.read_csv('./remastered.csv')

#data = pd.read_csv('./imputed_data_with_salePrice.csv')

#test_data = pd.read_csv('./test_imputed_data_without_salePrice.csv')



def alpha_calculation(price_origin_train, X_post_PCA, imputed_data, listype):
    #assign values to X,y:
    data_X = imputed_data.drop('SalePrice',axis = 1)
    data_y = imputed_data['SalePrice']
    log_y = np.log(data_y)
    
    #PCA transformation:
    #redo_data_X = scaler.fit(data_X).transform(data_X)
    #trans_redo_data_X = pca.fit_transform(redo_data_X)
    
    #find index 
    index = list(price_origin_train[price_origin_train['SaleCondition'] == listype].index)
    #print(index)
    
    #find the relevant values corresponding to the index
    trans_redo_data_X_dataframe = pd.DataFrame(X_post_PCA).ix[index]
    
    #print(remastered_train,train_X_after_PCA, train_data)
    
    #prediction
    pred = xgboost.predict(trans_redo_data_X_dataframe.values)
    
    #find the alpha corresponding to the abnormal salescondition
    attribute_alpha = log_y.ix[index].values.T/pred
    
    alpha = attribute_alpha.mean()
    origin_mse = mean_squared_error(log_y.ix[index].values.T,pred.reshape(-1,1))
    modified_mse = mean_squared_error(log_y.ix[index].values.T,alpha*pred.reshape(-1,1))
    ratio = (origin_mse-modified_mse)/origin_mse
    return alpha,origin_mse,modified_mse,ratio

In [420]:
#family alpha
fa_alpha,fa_origin_mse,fa_modified_mse,fa_ratio = alpha_calculation(remastered_train,train_X_after_PCA, train_data,'Family')
#abnormal
abn_alpha,abn_origin_mse,abn_modified_mse,abn_ratio = alpha_calculation(remastered_train,train_X_after_PCA, train_data,'Abnorml')
#
adj_alpha,adj_origin_mse,adj_modified_mse,adj_ratio = alpha_calculation(remastered_train,train_X_after_PCA, train_data,'AdjLand')
#
all_alpha,all_origin_mse,all_modified_mse,all_ratio = alpha_calculation(remastered_train,train_X_after_PCA, train_data,'Alloca')

par_alpha,par_origin_mse,par_modified_mse,par_ratio = alpha_calculation(remastered_train,train_X_after_PCA, train_data,'Partial')


/Users/hongzhengnan/anaconda3/envs/dme/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/Users/hongzhengnan/anaconda3/envs/dme/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/Users/hongzhengnan/anaconda3/envs/dme/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
/Users/hongzhengnan/anaconda3/envs/dme/lib/

In [421]:
print('alpha: %f origin: %f modified: %f improve %f'%(fa_alpha,fa_origin_mse,fa_modified_mse,fa_ratio))

alpha: 0.997221 origin: 0.019694 modified: 0.018603 improve 0.055366


In [422]:
print('alpha: %f origin: %f modified: %f improve %f'%(abn_alpha,abn_origin_mse,abn_modified_mse,abn_ratio))

alpha: 0.995661 origin: 0.020063 modified: 0.017653 improve 0.120106


In [423]:
print('alpha: %f origin: %f modified: %f improve %f'%(adj_alpha,adj_origin_mse,adj_modified_mse,adj_ratio))

alpha: 0.994099 origin: 0.013494 modified: 0.009093 improve 0.326145


In [424]:
print('alpha: %f origin: %f modified: %f improve %f'%(all_alpha,all_origin_mse,all_modified_mse,all_ratio))

alpha: 1.001239 origin: 0.019936 modified: 0.019547 improve 0.019526


In [425]:
print('alpha: %f origin: %f modified: %f improve %f'%(par_alpha,par_origin_mse,par_modified_mse,par_ratio))

alpha: 1.001762 origin: 0.011279 modified: 0.010763 improve 0.045758


# test

In [426]:
test_data = pd.read_csv('./new_test.csv')

In [427]:
for i in test_data.columns:
    if 'SaleCondition' in i:
        test_data.drop(i,axis = 1,inplace = True)

In [428]:
test_data = transform(test_data)

In [429]:
# test_data.drop(droplist,axis = 1,inplace = True)

In [430]:
pca_train.components_.shape

(50, 203)

In [431]:
test_data.shape

(1459, 203)

In [432]:
test_scaled = scaler.fit(test_data).transform(test_data)

In [433]:
test_data_post_PCA = np.dot(test_scaled,pca_train.components_.T)

In [434]:
test_data_post_PCA

array([[ 0.6839002 , -1.9485703 ,  5.24210018, ...,  1.2719571 ,
         0.19370372,  0.17669496],
       [ 0.14168803, -0.87539182,  0.90778597, ...,  0.99270753,
        -0.04312655,  0.17644379],
       [-0.68266589,  0.66982094,  0.34929717, ...,  0.33769595,
         0.10540486, -0.10006859],
       ...,
       [ 0.36446868,  0.93724503,  2.91496938, ..., -0.2816359 ,
         0.04075716,  0.6642519 ],
       [ 2.57181685, -2.93473002, -0.12131872, ...,  0.66886038,
         0.74322228, -0.04412603],
       [-2.55893976,  1.74259632,  0.29464248, ..., -0.19973046,
        -0.36985094, -0.52276327]])

In [435]:
pred_without_alpha = xgboost.predict(test_data_post_PCA)

In [436]:
alpha_dic = {'Normal':1,'Abnorml':abn_alpha,'AdjLand':adj_alpha,'Alloca':all_alpha,'Family':fa_alpha,'Partial':par_alpha}

In [437]:
test_s = pd.read_csv('./test.csv')

In [438]:
sale_condition = test_s.SaleCondition

In [439]:
pred_with_alpha = pred_without_alpha

In [440]:
for i in range(len(sale_condition)):
    pred_with_alpha[i] = alpha_dic[sale_condition[i]]*pred_without_alpha[i]

In [441]:
pd.DataFrame(np.exp(pred_with_alpha)).to_csv('./pred_with_alpha.csv',index=True)